In [2]:
from CIRESA import stereo_a, omni, solo, psp, CIR_detector, plot_heliosphere
import pandas as pd

# Define the start and end dates
start_date = '2022-01'
#start_date = '2019-11'
end_date = '2022-12'

# Generate a date range with monthly frequency
months = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m').tolist()

df = psp.load(months)
#omni.plot_omni(omni_df)
CIR = CIR_detector.plot_and_choose_CIR(df)
plot_heliosphere.plot_CIR_carrington(CIR)

C:/Users/14milosi/CIRESA/reduced_data/psp\psp_data2022-10.parquet


In [ ]:
from CIRESA import stereo_a, omni, solo, psp, CIR_detector, plot_heliosphere

#CIR = CIR_detector.plot_and_choose_CIR(df)
#CIR = CIR_detector.load_CIR(month='2022-10')
plot_heliosphere.plot_CIR_carrington(CIR)
#CIR_detector.save_to_CIR_data_base(CIR, 'psp')

c:\Users\14milosi\CIRESA\CIRESA\plot_heliosphere.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spacecraft['CARR_LON_RAD'] = spacecraft['CARR_LON']/180*3.14159


In [ ]:
from CIRESA import solo, propagation, plot_heliosphere, psp
import pandas as pd
import matplotlib
matplotlib.use('TKAgg')

solo_df = solo.load('2022-10')
psp_df = psp.load('2022-10')
omni_df = omni.load('2022-10')
df = pd.concat([solo_df, psp_df])
sim = propagation.ballistic(
    omni_df['2022-10-07':'2022-10-15'], cadence='0.5H')
plot_heliosphere.plot_spacecraft_carrington(omni_df['2022-10-07':'2022-10-15'])


C:/Users/14milosi/CIRESA/reduced_data/solo\solo_data2022-10.parquet
C:/Users/14milosi/CIRESA/reduced_data/psp\psp_data2022-10.parquet
C:/Users/14milosi/CIRESA/reduced_data/omni\omni_data2022-10.parquet


In [6]:
#plt.plot(df2['CARR_LON'])
#plt.show()
df2.to_parquet('reduced_data\psp\psp_data'+'2022-09'+'.parquet')

In [ ]:
from CIRESA import stereo_a, omni, solo, psp, CIR_detector, plot_heliosphere, get_coordinates
import pandas as pd
import matplotlib.pyplot as plt

# Define the start and end dates
start_date = '2022-09'
#start_date = '2019-11'
end_date = '2022-11'

# Generate a date range with monthly frequency
months = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m').tolist()

df2 = psp.load(months)

# GET COORDINATES

import pandas as pd
import numpy as np
from datetime import timedelta

# Assuming `df2` contains data for the entire month, indexed by timestamps

# Loop through each day in the month
for day in pd.date_range(start='2022-09-01', end='2022-09-30', freq='D'):
    
    # Select data for the current day
    day_df = df2[(df2.index >= day) & (df2.index < day + timedelta(days=1))]
    
    # Resample and process the data as needed
    coord_df = day_df.resample(rule='6H').median()
    print(len(day_df), len(coord_df))
    # Assuming get_coordinates.get_coordinates is your function for getting coordinate values
    carr_lons, psp_r, psp_lats, psp_lon = get_coordinates.get_coordinates(coord_df, 'PSP')

    coord_df['CARR_LON'] = carr_lons
    coord_df['CARR_LON_RAD'] = coord_df['CARR_LON'] / 180 * np.pi
    coord_df['LAT'] = psp_lats

    psp_lon = np.asarray(psp_lon)
    if (psp_lon < -175).any() & (psp_lon > 175).any():
        psp_lon[psp_lon < 0] += 360

    coord_df['INERT_LON'] = psp_lon
    #print(psp_lon)

    # Reindex and interpolate the resampled dataframe
    coord_df = coord_df.reindex(day_df.index).interpolate(method='linear')
    print(len(day_df), len(coord_df), len(df2.loc[coord_df.index]))
    # Updating the original dataframe
    df2.loc[coord_df.index, 'CARR_LON'] = get_coordinates.calculate_carrington_longitude_from_lon(coord_df.index, coord_df['INERT_LON'])
    df2['CARR_LON_RAD'] = df2['CARR_LON'] / 180 * np.pi
    df2['LAT'] = coord_df['LAT'].copy()
    df2['INERT_LON'] = coord_df['INERT_LON'].copy()

plt.plot(df2['INERT_LON'])
plt.show()

In [2]:
#PLOT MOVIE

from CIRESA import stereo_a, omni, solo, psp, CIR_detector, plot_heliosphere
import pandas as pd
# Define the start and end dates
start_date = '2022-10'
#start_date = '2019-11'
end_date = '2023-01'

# Generate a date range with monthly frequency
months = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m').tolist()

#months = '2022-10'

df = solo.load(months)
df4 = omni.load(months)
df2 = psp.load(months)
df3 = stereo_a.load(months)
CIRS = CIR_detector.load_CIR(months)
plot_heliosphere.progressive_model_movie([df, df2, df3, df4]
                                         , plot_cadence = 24, persistance= 7
                                         , directory= 'FIRSTELASTIC'
                                         , sim_cadence='1H'
                                         , HEE = True
                                         , back_prop=False
                                         , model = 'inelastic'
                                         , movie= True
                                         , rlim = 1.5
                                         , COR=1)

# plot_heliosphere.progressive_model_movie([df, df2, df3, df4]
#                                          , plot_cadence = 24, persistance= 7
#                                          , directory= 'FIRSTBALLISTIC'
#                                          , sim_cadence='1H'
#                                          , HEE = True
#                                          , back_prop=False
#                                          , model = 'ballistic'
#                                          , movie= True
#                                          , rlim = 1.5)


C:/Users/14milosi/CIRESA/reduced_data/solo\solo_data2022-10.parquet
C:/Users/14milosi/CIRESA/reduced_data/solo\solo_data2022-11.parquet
C:/Users/14milosi/CIRESA/reduced_data/solo\solo_data2022-12.parquet
C:/Users/14milosi/CIRESA/reduced_data/omni\omni_data2022-10.parquet
C:/Users/14milosi/CIRESA/reduced_data/omni\omni_data2022-11.parquet
C:/Users/14milosi/CIRESA/reduced_data/omni\omni_data2022-12.parquet
C:/Users/14milosi/CIRESA/reduced_data/psp\psp_data2022-10.parquet
C:/Users/14milosi/CIRESA/reduced_data/psp\psp_data2022-11.parquet
C:/Users/14milosi/CIRESA/reduced_data/stereo_a\stereo_a_data2022-10.parquet
C:/Users/14milosi/CIRESA/reduced_data/stereo_a\stereo_a_data2022-11.parquet
C:/Users/14milosi/CIRESA/reduced_data/stereo_a\stereo_a_data2022-12.parquet
Plot 1 out of 84
Plot 2 out of 84
Plot 3 out of 84
Plot 4 out of 84
Plot 5 out of 84
Plot 6 out of 84
Plot 7 out of 84
Plot 8 out of 84
Plot 9 out of 84
Plot 10 out of 84
Plot 11 out of 84
Plot 12 out of 84
Plot 13 out of 84
Plot 14

In [ ]:
#PLOT MOVIE

from CIRESA import stereo_a, omni, solo, psp, CIR_detector, plot_heliosphere
import pandas as pd
# Define the start and end dates
start_date = '2022-08'
#start_date = '2019-11'
end_date = '2023-01'

# Generate a date range with monthly frequency
months = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m').tolist()

#months = '2022-10'

df = solo.load(months)
df2 = omni.load(months)
df3 = psp.load(months)
df4 = stereo_a.load(months)
#CIRS = CIR_detector.load_CIR(months)
#plot_heliosphere.progressive_model_movie([df, df2, df3, df4]
 #                                        , hours = 4, movie= True, n= 5
  #                                       , directory= 'modelmovie6'
   #                                      , sim_cadence='1H'
    #                                     , inertial = False)
plot_heliosphere.plot_n_days(pd.concat([df, df2, df3, df4])
                                         , plot_cadence = 4, movie= True
                                         , persistance= 5
                                         , directory= 'modelmovie9'
                                         )

In [1]:
from CIRESA import stereo_a, omni, solo, psp

import pandas as pd

# Define the start and end dates
start_date = '2022-08'
#start_date = '2019-11'
end_date = '2023-01'

# Generate a date range with monthly frequency
months = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m').tolist()


#omni.download_reduce_save_space(month=months, cadence='0.1H')
#stereo_a.download_reduce_save_space(month=months, cadence='0.1H')
#solo.download_reduce_save_space(month=months, cadence='0.1H')
psp.download_reduce_save_space(month=months, cadence='0.1H')


2022-08
C:/Users/14milosi/CIRESA/reduced_data/psp\psp_data2022-08.parquet
Deleting: []
2022-09
C:/Users/14milosi/CIRESA/reduced_data/psp\psp_data2022-09.parquet


c:\Users\14milosi\CIRESA\CIRESA\psp.py:228: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psp_df['polarity'] = ['+' if pol > 0 else '-' for pol in psp_df['POL']]


Deleting: []
2022-10
C:/Users/14milosi/CIRESA/reduced_data/psp\psp_data2022-10.parquet
Deleting: []
2022-11
C:/Users/14milosi/CIRESA/reduced_data/psp\psp_data2022-11.parquet
Deleting: []
2022-12


12-Sep-24 12:15:03: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_rtn/2022/


### DOWNLOADING MAGNETIC FIELD DATA: 1 out 3 INSTRUMENTS ###


12-Sep-24 12:15:05: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_rtn/2022/psp_fld_l2_mag_rtn_2022120100_v02.cdf to psp_data/fields/l2/mag_rtn/2022/psp_fld_l2_mag_rtn_2022120100_v02.cdf
12-Sep-24 12:15:12: Download complete: psp_data/fields/l2/mag_rtn/2022/psp_fld_l2_mag_rtn_2022120100_v02.cdf
12-Sep-24 12:15:13: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_rtn/2022/psp_fld_l2_mag_rtn_2022120106_v02.cdf to psp_data/fields/l2/mag_rtn/2022/psp_fld_l2_mag_rtn_2022120106_v02.cdf
12-Sep-24 12:15:21: Download complete: psp_data/fields/l2/mag_rtn/2022/psp_fld_l2_mag_rtn_2022120106_v02.cdf
12-Sep-24 12:15:22: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_rtn/2022/psp_fld_l2_mag_rtn_2022120112_v02.cdf to psp_data/fields/l2/mag_rtn/2022/psp_fld_l2_mag_rtn_2022120112_v02.cdf
12-Sep-24 12:15:30: Download complete: psp_data/fields/l2/mag_rtn/2022/psp_fld_l2_mag_rtn_2022120112_v02.cdf
12-Sep-24 12:15:30: Downloading https://spdf.gsfc.nasa.

In [9]:
from CIRESA import psp 

df = psp.load('2022-08')
df.drop_duplicates

C:/Users/14milosi/CIRESA/reduced_data/psp\psp_data2022-08.parquet


<bound method DataFrame.drop_duplicates of                              N      DENS         V_R        V_T        V_N  \
Time                                                                         
2022-08-01 00:00:00  10.476392  0.042388  407.346741 -53.435143 -60.774948   
2022-08-01 00:06:00  11.161665  0.056460  376.476074 -45.118725 -57.113003   
2022-08-01 00:12:00  11.338790  0.078054  394.589600 -49.720802 -54.693808   
2022-08-01 00:18:00  11.560798  0.065374  385.158112 -45.125340 -59.222851   
2022-08-01 00:24:00  10.753632  0.059910  363.983673 -44.973682 -50.601885   
...                        ...       ...         ...        ...        ...   
2022-08-30 23:30:00  57.578457  5.338610  397.412628 -57.562431 -30.150993   
2022-08-30 23:36:00  61.204845  4.639652  388.412506 -58.782310 -25.607592   
2022-08-30 23:42:00  64.144089  3.614267  442.976074 -72.707901 -38.574417   
2022-08-30 23:48:00  53.090363  2.748883  450.915833 -64.712677 -46.989803   
2022-08-30 23:54:00  

In [ ]:
from CIRESA import psp, psp, psp, get_coordinates
import numpy as np
import pandas as pd

# Define the start and end dates
start_date = '2022-10'
#start_date = '2019-11'
end_date = '2023-01'

# Generate a date range with monthly frequency
months = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m').tolist()

for m in months:
    psp_df = psp.load(month=m)
    psp_df['CARR_LON'] = get_coordinates.calculate_carrington_longitude_from_lon(psp_df.index, psp_df['INERT_LON'])
    psp_df['CARR_LON_RAD'] = psp_df['CARR_LON']/180*np.pi
    psp_df.to_parquet('reduced_data\psp\psp_data'+m+'.parquet')
